# Melting

**Source:** Wagner, T. J., Dell, R. W., & Eisenman, I. (2017). An Analytical Model of Iceberg Drift. *Journal of Physical Oceanography*.

## Melt Terms

### Wind-driven wave erosion:

\begin{equation}
M_e = 0.5 S_s = 0.75 \mid \vec{v_a} - \vec{v_w} \mid^{0.5} + 0.05 \mid \vec{v_a} - \vec{v_w} \mid
\end{equation}


### Turbulent basal melt:

\begin{equation}
M_b = 0.58 \mid \vec{v_w} - \vec{v_i} \mid^{0.8} (T_w - T_i)L^{-0.2}
\end{equation}


### Buoyant convection (thermal side-wall erosion):

\begin{equation}
M_v = 0.0076 T_w + 0.0013 T_w^2
\end{equation}


#### Notes:

- *Other processes such as surface melt, have been found to be small compared to these terms (Savage 2001).*
- *Water temperature, $T_w$, is approximated by the SST.*
- *Iceberg temperature, $T_i$, is assumed constant at $T_i=-4^\circ C$. (El Tahan et. al. 1987)*
- *All temperatures are in degrees celcius and L is in meters*
- *Each of the melt terms, $M-E$, $M_b$, and $M_v$, are expressed in meters of change in iceberg dimensions per day*
- *We assume that these processes are linearly additive, such that iceberg volume evolves as:*

\begin{equation}
\frac{dV}{dt} = \frac{d(LWH)}{dt}
\end{equation}


## Melt Rates

$$\frac{dL}{dt} = \frac{dW}{dt} = -M_v - M_e$$

$$\frac{dH}{dt} = -M_b$$

$$L^{i+1} = L^{i} + \frac{dL}{dt}$$

$$W^{i+1} = W^{i} + \frac{dW}{dt}$$

$$H^{i+1} = H^{i} + \frac{dH}{dt}$$

## Rollovers

### Aspect ratio:

$$\epsilon \equiv \frac{W}{H}$$

### Critical value:

$$\epsilon_c = \sqrt{6\frac{\rho_i}{\rho_w}(1 - \frac{\rho_i}{\rho_w})}$$

*below which a rectangular iceberg is unconditionally unstable.* 

#### Assumptions:
*Assuming $\rho_i = 850 kg/m^3$ (based on tabular density for icebergs in the Southern Ocean) and a water density of $\rho_w = 1025 kg/m^3$ we get:*

$$\frac{\rho_i}{\rho_w} = 0.83$$

$$\epsilon_c = 0.92$$

In [5]:
def melting(i):
    
    I = i
    
    # Melt terms
    
    Me = CMe1 * (Cs1 * Ua^Cs2 + Cs3 * Ua)  ## Wind driven erosion
    Mv = CMv1 * SST + CMv2 * SST^2  ## Thermal side wall erosion 
    Mb = CMb1 * np.sqrt((ui - uw)^2 + (vi - vw)^2)^CMb2 * (SST - Ti0) / l(I)^CMb3  ## Turbulent basal melt 

    # Melt rates
    
    dldt = - Mv - Me 
    dhdt = - Mb
    l[I+1] = l[I] + dldt * Dt  
    w[I+1] = w[I] + dldt * Dt  
    h[I+1] = h[I] + dhdt * Dt 
    
    
    # Check if iceberg size is negative
    
    if l[I+1] < 0 | w[I+1] < 0 | h[I+1] < 0:
        l[I+1] = 0 
        w[I+1] = 0 
        h[I+1] = 0
        melted = 1   ## Boolean
        mm = mm + 1   ## Counter for icebergs that have melted
    
    
    # Rollover
    
    if w[I+1] < 0.85 * h[I+1]:
        hn = w[I+1]  ## new height
        w[I+1] = h[I+1] 
        h[I+1] = hn
    
    
    # Check if length is greater than width
    
    if w[I+1] > l[I+1]:
        wn = l[I+1] 
        l[I+1] = w[I+1] 
        w[I+1] = wn
    
    
    # New volume and change in volume (dv)
    
    v[I+1] = l[I+1] * w[I+1] * h[I+1]
    dv[I+1] = v(I) - v[I+1]
    
    
    # Check if iceberg survived
    
    if I == lt-1 & v[I+1] > 0:
        ss = ss + 1   ## counter for icebergs that have survived
    
    
    # Store melt rates
    
    Mev = Me 
    Mvv = Mv 
    Mbv = Mb
    

In [ ]:
from scipy.spatial import cKDTree as ckd


def drifting(i, LAT, LON, yil, xil, tts, uaF, vaF, uwF, vwF, sst, Ut, S, ui, vi, dtR, minLON, maxLON, minLAT, maxLAT, j):
    
    I = i
    
    # Find nearest neighbour
    ## CAUTION: this only works on a rectangular grid!
    
    YI = ckd(LAT, yil[I])
    XI = ckd(LON, xil[I])
    
    
    # Interpolate fields linearly between timesteps
    
    timestep = tt[tts + I]
    t1  = floor(timestep) 
    t2 = t1 + 1
    dt1 = timestep - t1 
    dt2 = t2 - timestep
    ua = uaF[XI,YI,t1] * dt1 + uaF[XI,YI,t2] * dt2 
    va = vaF[XI,YI,t1] * dt1 + vaF[XI,YI,t2] * dt2 
    uw = uwF[XI,YI,t1] * dt1 + uwF[XI,YI,t2] * dt2 
    vw = vwF[XI,YI,t1] * dt1 + vwF[XI,YI,t2] * dt2 
    SST= sst[XI,YI,t1] * dt1 + sst[XI,YI,t2] * dt2 
    
    
    # Compute wind speed and "U tilde" at location for a given iceberg size
    
    Ua = np.sqrt(ua^2+va^2)
    UT = Ut(Ua, yil[I], S(l[I],w[I]))  ## Ut and S are functions
    
    
    # Iceberg translation
    ## Note the conversion from meters to degrees lon/lat
    
    dlon = ui * dtR 
    dlat = vi * dtR 

    uiv(I) = ui 
    viv(I) = vi
    
    uav(I) = ua 
    vav(I) = va
    
    uwv(I) = uw 
    vwv(I) = vw
    
    temp(I) = SST

    yil[I+1] = yil[I] + dlat 
    xil[I+1] = xil[I] + dlon / cos((yil[I+1] + yil[I]) / 2*pi / 180) 
    
    
    # Check if out-of-bounds
    ## TODO: find replacements for MATLAB's find() and any() functions in Python
    
    if xil[I+1] > maxLON | xil[I+1] < minLON | yil[I+1] > maxLAT | yil[I+1] < minLAT:
    
        outofbound = 1
        ob = ob + 1
        print('iceberg {} left boundary at timestep {} \n'.format(j, I))

        
    ## now check you didn't send the iceberg on land

    else:  
        
        yi2[1] = find(LAT <= yil[I+1], 1, 'last')
        yi2[2] = find(LAT > yil[I+1], 1, 'first')
        xi2[1] = find(LON <= xil[I+1], 1, 'last')
        xi[2] = find(LON > xil[I+1], 1, 'first')
        
        if any(find(msk(xi2, yi2) == 0)):
            
            yil[I+1] = yil[I]   ## i.e. when I get put within one grid box of land
            xil[I+1] = xil[I]   ## I assume the iceberg don't move, until it doesn't happen anymore
    

In [19]:
import numpy as np

def a(U):
    # \alpha in the paper
    a = np.sqrt(2) / np.power(U, 3) * (1 - np.sqrt(1 + np.power(U, 4)))
    return a

def b(U):
    # \beta in the paper
    b = np.real(1 / np.power(U, 3) * np.sqrt(1 + np.power(U, 4)) - 3 * np.power(U, 4) - 4)
    return b

def ff(lati):
    # Latitude in degrees
    ff = 2 * om * sin(abs(lati) * pi/180)
    return ff

def S(l, w):
    # Harmonic mean length
    S = pi * l * w / (l + w)
    return S

def Ut(u, lati, S):
    # \Lambda in the papers
    Ut = np.sqrt(2) * Cw * g / np.multiply(ff(lati), u/s)
    return Ut


In [31]:
# def analytic_parameters():
    
R = 6378 * 1e3  ## earth radius in m
rhow = 1027  ## density of water (kg/m^3)
rhoa = 1.2  ## density of air   (kg/m^3)
rhoi = 850  ## density of ice   (kg/m^3)
drho = rhow - rhoi
Cw = 0.9  ## bulk coefficient water  (Bigg et al 1997)
Ca = 1.3  ## bulk coefficient air    (Bigg et al 1997)
om = 7.2921 * 1e-5  ## rotation rate of earth (rad/s)
g = np.sqrt(rhoa * drho / rhow / rhoi * (Ca / Cw))  ## gamma = np.sqrt(ca/cw)


# Melt parameters

Ti0 = -4
Cs1 = 1.5 
Cs2 = 0.5 
Cs3 = 0.1
CMv1 = 7.62e-3 
CMv2 = 1.29e-3
CMe1 = 0.5
CMb1 = 0.58 
CMb2 = 0.8 
CMb3 = 0.2
    

In [25]:
import numpy.matlib

def construct_seeding():
    
    glacier = 'H'   #pick the glacier you want to construct the seeding for

    if glacier == 'H':
        # Helheim Ice Sheet Seeding Locations ---------------------------------
        Glac_Y = np.arange(259,262,1)  #[261,190] 66.3500 N, 38.2000 W coord, Helheim
        Glac_X = np.arange(188,192,1) 
        glac_name = 'Helheim'
        
    elif glacier == 'J':
        # Jakobshaven Ice Sheet Seeding Locations -----------------------------
        Glac_Y = np.arange(276,280,1)  #[277,120] 69 10 N 49 50 W coord, Jakobshavn
        Glac_X = np.arange(119,123,1) 
        glac_name = 'Jakobsh'
        
    elif glacier == 'K':
        # Kangerd Ice Sheet Seeding Locations ---------------------------------
        Glac_Y = np.arange(267,271,1)  #[277,120] 68 38 N 33 0 W coord, Kangerd
        Glac_X = np.arange(213,1,218) 
        glac_name = 'Kangerd'
        
    elif glacier == 'L': 
        # Laurentide Ice Sheet Seeding Locations ------------------------------
        Glac_Y = np.arange(245,255,2)  #Laurentide LAT seed locations (in ECCO 2 grid)
        Glac_X = np.arange(86,101,2)    #Laurentide LON seed locations (in ECCO 2 grid)
        glac_name = 'Laurent' 
    

    seed_LAT = [LAT[Glac_Y[0]]*[1, 1], LAT[Glac_Y[-1]]*[1, 1]] 
    seed_LON = np.matlib.repmat([LON[Glac_X[0]], LON[Glac_X[-1]]],[2,1]) 
    seed_LAT = np.matlib.repmat([LAT[Glac_Y[0]], LAT[Glac_Y[-1]]],[2,1]) 
    seed_LON = [LON[Glac_X[0]]*[1, 1], LON[Glac_X[-1]]*[1, 1]] 
    Seed_Y, Seed_X = meshgrid(Glac_Y, Glac_X) 
        

In [28]:
import time

def tic():
    #Homemade version of matlab tic and toc functions
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()

def toc():
    import time
    if 'startTime_for_tictoc' in globals():
        print("Elapsed time is {} seconds.".format(str(time.time() - startTime_for_tictoc)))
    else:
        print("Toc: start time not set")
        
# Source: http://stackoverflow.com/questions/5849800/tic-toc-functions-analog-in-python/5849861
# Credit: GuestUser

In [ ]:
import scipy.io as sio

def iceberg_shell():
    
    # Set directories
    
    modelfull = 'ECCO_20th'
    modelshort = 'E2'
    root = '/home/evankielley/WagnerModel'
    condloc = root '/conditions' modelfull '/'
    outloc = root '/output' modelfull '/'
    
    
    # Load input fields
    
    tic()
    mask = sio.loadmat(condloc 'mask.mat')
    E2_vels_1992 = sio.loadmat(condloc modelshort '_vels_1992.mat')
    E2_sst_1992 = sio.loadmat(condloc modelshort '_sst_1992.mat')
    print('Model data loaded \n')
    toc()
    
    
    # Read in all analytic parameters
    ## Done by setting them as global vars
    
    
    # Specify the space domain
    
    
    
    
    
    
    
    